<a href="https://colab.research.google.com/github/Ayush-Tibrewal/CongnitiveLoad-Detection/blob/main/MAT_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ls /content/drive/MyDrive/Signal_Processing/data_set_eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0

data_csv/        Subject09_1.edf  Subject18_2.edf  Subject28_1.edf
Subject00_1.edf  Subject09_2.edf  Subject19_1.edf  Subject28_2.edf
Subject00_2.edf  Subject10_1.edf  Subject19_2.edf  Subject29_1.edf
Subject01_1.edf  Subject10_2.edf  Subject20_1.edf  Subject29_2.edf
Subject01_2.edf  Subject11_1.edf  Subject20_2.edf  Subject30_1.edf
Subject02_1.edf  Subject11_2.edf  Subject21_1.edf  Subject30_2.edf
Subject02_2.edf  Subject12_1.edf  Subject21_2.edf  Subject31_1.edf
Subject03_1.edf  Subject12_2.edf  Subject22_1.edf  Subject31_2.edf
Subject03_2.edf  Subject13_1.edf  Subject22_2.edf  Subject32_1.edf
Subject04_1.edf  Subject13_2.edf  Subject23_1.edf  Subject32_2.edf
Subject04_2.edf  Subject14_1.edf  Subject23_2.edf  Subject33_1.edf
Subject05_1.edf  Subject14_2.edf  Subject24_1.edf  Subject33_2.edf
Subject05_2.edf  Subject15_1.edf  Subject24_2.edf  Subject34_1.edf
Subject06_1.edf  Subject15_2.edf  Subject25_1.edf  Subject34_2.edf
Subject06_2.edf  Subject16_1.edf  Subject25_2.edf  Subject35_1

In [ ]:
data_path = '/content/drive/MyDrive/Signal_Processing/data_set_eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/'

In [ ]:
pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 19.8 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import numpy as np
import mne
import pylab as pl

edf_path = "raw_data"
csv_path = "data_csv"
done_list = []

def to_csv(file):
    sub_edf = mne.io.read_raw_edf(os.path.join(data_path, file))
    header = ','.join(sub_edf.ch_names)
    np.savetxt(os.path.join(csv_path, file.split(".")[0] + ".csv"),
                sub_edf.get_data().T, delimiter=',', header=header)
    done_list.append(file)
    print(file, "done")

files = os.listdir("/content/drive/MyDrive/Signal_Processing/data_set_eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/")
for file in files:
    if file[-4:] == 'edf':
        to_csv(files)
print(done_list, sep="\n")

[]


In [ ]:
import pandas as pd
import numpy as np

import os
from tqdm.notebook import tqdm
from sklearn.decomposition import FastICA
from scipy.signal import butter, lfilter, freqz, hilbert
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
LOWCUT = 0.5
HIGHCUT = 30
SAMPLING_FREQUENCY = 500

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(eeg, lowcut, highcut, fs, order=5):
    # Takes raw eeg and returns filtered EEG data
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, eeg)
    return y

def ICA(signal):
    # Takes a raw eeg and returns raw EEG with artefacts removed using ICA
    eeg = np.stack([signal]).T
    ica = FastICA()
    ica.fit(eeg)
    components = ica.transform(eeg)
    restored = ica.inverse_transform(components)
    signal = restored.T[0]

    return signal

# Helper Functions

def df_filter(df, cols):
    # tqdm() enables a progress bar over an iterable
    for ch in tqdm(cols[:]):
        df[ch] = butter_bandpass_filter(df[ch], LOWCUT, HIGHCUT, SAMPLING_FREQUENCY)
    return df

def df_ica(df, cols):
    for ch in tqdm(cols[:]):
        df[ch] = ICA(df[ch])
    return df

In [ ]:
# EEG Channels
CHANNELS = ["AF3", "F7", "F3", "FC5", "T7", "P7", "O1", "O2", "P8", "T8", "FC6", "F4", "F8", "AF4"]
CHANNELS1 = ["AF7"]
# EEG Data
# data_dic = {}
# for file in os.listdir("./data_csv"):
#     sub = int(file.split('.')[0]) # Subject no.
#     data_dic[sub] = os.path.join(data_path, file)

# SUB = list(df_log.index.values)

In [ ]:
file = 'Subject01_1.csv'
sampling_freq = 128
sampling_duration = 329
number_of_samples = sampling_freq * sampling_duration

time = np.linspace(0, sampling_duration, number_of_samples, endpoint=False)
# print("Subject : " + str(sub))
data = pd.read_csv('/content/drive/MyDrive/Signal_Processing/data_set_eeg-during-mental-arithmetic-tasks-1.0.0/eeg-during-mental-arithmetic-tasks-1.0.0/' + file)

# Selecting Timestamp and Raw EEG signals only
print("\tSelecting Timestamp and Raw EEG signals only: ")
df = data[CHANNELS + ['# COUNTER']]
#df = df[raw_cols + ['Timestamp']]

# Filtering the Raw EEG signals
print("\tFiltering the raw eeg signals:")
df = df_filter(df, CHANNELS)

# Applying ICA for artefact removal
print("\tApplying ICA for artefact removal:")
df = df_ica(data, CHANNELS)

# Storing Data
df.to_csv('/content/drive/MyDrive/Signal_Processing/preprocessed_data/' + file + 'ica_butter')

FileNotFoundError: ignored